In [1]:
using Revise
using Laplacians
using Pkg
@show Pkg.pathof(Laplacians)
using SparseArrays
using LinearAlgebra

Pkg.pathof(Laplacians) = "/Users/rjkyng/Dropbox/Repos/Laplacians.jl/src/Laplacians.jl"


In [2]:
function indexToLinear(x,y,z,xlen,ylen)
    return x+(y-1)*xlen+(z-1)*xlen*ylen
end

indexToLinear (generic function with 1 method)

In [12]:
s = 102

s1 = s
s2 = s
s3 = s
b1,b2,b3 = 3,3,3
w = 1e7
@time A = ggrid3_checkered(s1,s2,s3,b1,b2,b3,w);
x,y,z = ggrid3coords(s1,s2,s3);

n = s1*s2*s3
zsidesize = s1*s2
xsidesize = s2*s3
ysidesize = s1*s3
zside1b = 1:zsidesize # bot 
zside2b = n-zsidesize+1:n # top

zside1 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside1[i] = indexToLinear(x,y,1,s1,s2)
end

zside2 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside2[i] = indexToLinear(x,y,s3,s1,s2)
end

xside1 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
#     @show (i,(y,z))
#     if (y == 2 && z == 2)
#         @show z == 2
#         @show indexToLinear(1,y,z,s1,s2)
#     end
    xside1[i] = indexToLinear(1,y,z,s1,s2)
end

xside2 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
    xside2[i] = indexToLinear(s1,y,z,s1,s2)
end

yside1 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside1[i] = indexToLinear(x,1,z,s1,s2)
end

yside2 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside2[i] = indexToLinear(x,s2,z,s1,s2)
end

bndry = [zside1; zside2; xside1; xside2; yside1; yside2];
@time int = setdiff(1:n,bndry)

ni = length(int) #this is the remaining variables
@assert(ni == (s-2)^3)

@time L = lap(A) # create the Laplacian
@time M = L[int,int];
# remove the rows and colums corresponding to the boundary of the grid
# -- this is because we essentially want to force boundary voltages to be zero



  0.552202 seconds (48 allocations: 353.147 MiB, 42.09% gc time)
  0.215104 seconds (26 allocations: 26.643 MiB, 51.96% gc time)
  0.117319 seconds (22 allocations: 160.976 MiB, 8.18% gc time)
  0.193535 seconds (13 allocations: 121.622 MiB, 42.70% gc time)


In [13]:
b = randn(ni);

In [17]:
@time sol = approxchol_sddm(M);
@time x = sol(b,tol=1e-8)
norm(M*x-b)/norm(b)

  7.454572 seconds (7.94 M allocations: 1.884 GiB, 28.04% gc time)
  7.131198 seconds (260 allocations: 801.102 MiB, 6.40% gc time)


2.7279549749195836e-8

# more blocks

In [19]:
s = 102

s1 = s
s2 = s
s3 = s
b1,b2,b3 = 10,10,10
w = 1e7
@time A = ggrid3_checkered(s1,s2,s3,b1,b2,b3,w);
x,y,z = ggrid3coords(s1,s2,s3);

n = s1*s2*s3
zsidesize = s1*s2
xsidesize = s2*s3
ysidesize = s1*s3
zside1b = 1:zsidesize # bot 
zside2b = n-zsidesize+1:n # top

zside1 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside1[i] = indexToLinear(x,y,1,s1,s2)
end

zside2 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside2[i] = indexToLinear(x,y,s3,s1,s2)
end

xside1 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
#     @show (i,(y,z))
#     if (y == 2 && z == 2)
#         @show z == 2
#         @show indexToLinear(1,y,z,s1,s2)
#     end
    xside1[i] = indexToLinear(1,y,z,s1,s2)
end

xside2 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
    xside2[i] = indexToLinear(s1,y,z,s1,s2)
end

yside1 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside1[i] = indexToLinear(x,1,z,s1,s2)
end

yside2 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside2[i] = indexToLinear(x,s2,z,s1,s2)
end

bndry = [zside1; zside2; xside1; xside2; yside1; yside2];
@time int = setdiff(1:n,bndry)

ni = length(int) #this is the remaining variables
@assert(ni == (s-2)^3)

@time L = lap(A) # create the Laplacian
@time M = L[int,int];
# remove the rows and colums corresponding to the boundary of the grid
# -- this is because we essentially want to force boundary voltages to be zero


  0.594877 seconds (48 allocations: 353.147 MiB, 42.61% gc time)
  0.288612 seconds (26 allocations: 26.643 MiB, 51.72% gc time)
  0.159417 seconds (22 allocations: 160.976 MiB, 7.63% gc time)
  0.258484 seconds (13 allocations: 121.622 MiB, 38.18% gc time)


In [20]:

b = randn(ni);

In [21]:
@time sol = approxchol_sddm(M);
@time x = sol(b,tol=1e-8)
norm(M*x-b)/norm(b)

  6.541223 seconds (7.94 M allocations: 1.884 GiB, 33.61% gc time)
  7.022960 seconds (296 allocations: 938.434 MiB, 7.05% gc time)


2.9982095652677284e-8

# even more blocks

In [22]:
s = 102

s1 = s
s2 = s
s3 = s
b1,b2,b3 = 25,25,25
w = 1e7
@time A = ggrid3_checkered(s1,s2,s3,b1,b2,b3,w);
x,y,z = ggrid3coords(s1,s2,s3);

n = s1*s2*s3
zsidesize = s1*s2
xsidesize = s2*s3
ysidesize = s1*s3
zside1b = 1:zsidesize # bot 
zside2b = n-zsidesize+1:n # top

zside1 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside1[i] = indexToLinear(x,y,1,s1,s2)
end

zside2 = Array{Int64}(undef,zsidesize)
for (i,(x,y)) in Base.Iterators.enumerate(Base.product(1:s1,1:s2))
    zside2[i] = indexToLinear(x,y,s3,s1,s2)
end

xside1 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
#     @show (i,(y,z))
#     if (y == 2 && z == 2)
#         @show z == 2
#         @show indexToLinear(1,y,z,s1,s2)
#     end
    xside1[i] = indexToLinear(1,y,z,s1,s2)
end

xside2 = Array{Int64}(undef,xsidesize)
for (i,(y,z)) in Base.Iterators.enumerate(Base.product(1:s2,1:s3))
    xside2[i] = indexToLinear(s1,y,z,s1,s2)
end

yside1 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside1[i] = indexToLinear(x,1,z,s1,s2)
end

yside2 = Array{Int64}(undef,ysidesize)
for (i,(x,z)) in Base.Iterators.enumerate(Base.product(1:s1,1:s3))
    yside2[i] = indexToLinear(x,s2,z,s1,s2)
end

bndry = [zside1; zside2; xside1; xside2; yside1; yside2];
@time int = setdiff(1:n,bndry)

ni = length(int) #this is the remaining variables
@assert(ni == (s-2)^3)

@time L = lap(A) # create the Laplacian
@time M = L[int,int];
# remove the rows and colums corresponding to the boundary of the grid
# -- this is because we essentially want to force boundary voltages to be zero


  0.519243 seconds (48 allocations: 353.147 MiB, 46.21% gc time)
  0.358532 seconds (26 allocations: 26.643 MiB, 60.86% gc time)
  0.118090 seconds (22 allocations: 160.976 MiB, 11.85% gc time)
  0.251864 seconds (13 allocations: 121.622 MiB, 40.07% gc time)


In [24]:
b = randn(ni);

In [25]:
@time sol = approxchol_sddm(M);
@time x = sol(b,tol=1e-8)
norm(M*x-b)/norm(b)

  6.680540 seconds (7.94 M allocations: 1.884 GiB, 33.23% gc time)
  7.658452 seconds (304 allocations: 968.952 MiB, 6.30% gc time)


3.281873561107211e-8